In [1]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from analysts_tools.redash_methods import *
from procurement_lib import redash, send_slack_notification, GoogleSheet

import random
import time
import datetime

In [2]:
df_grads = get_fresh_query_result("https://internal-redash.federate.frubana.com/",199393,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_fmg_comex = get_fresh_query_result("https://internal-redash.federate.frubana.com/",199394,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_kam = get_fresh_query_result("https://internal-redash.federate.frubana.com/",199396,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_fmg_br = get_fresh_query_result("https://internal-redash.federate.frubana.com/",199392,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200
200
200
200


In [3]:
def gmv(num):    
    query = """
    SELECT 
        s.identifier_value AS city,
        TO_CHAR(fs.order_close_date, 'YYYY')::int AS year,
        TO_CHAR(fs.order_close_date, 'MM')::int AS month,
        TO_CHAR(fs.order_close_date, 'WW')::int AS weekk,
        CASE
            WHEN fsd.adjustment_reason ILIKE '%kam%' THEN 'KAM'
            WHEN fsd.adjustment_reason ILIKE 'hoo%' THEN 'HOOK'
            WHEN fsd.adjustment_reason ILIKE '%\_NB\_%' THEN 'SUPER DISCOUNTS'
            WHEN fsd.adjustment_reason ILIKE 'sh\_%' THEN 'SUPER HOOKS'
            WHEN fsd.adjustment_reason ILIKE 'EXTRA\_INV%' THEN 'ADDITIONAL INVESTMENT'
            WHEN (fsd.adjustment_reason ILIKE '%merma%' OR fsd.adjustment_reason ILIKE '%acmk%' OR fsd.adjustment_reason ILIKE '%liquidation%' OR fsd.adjustment_reason IS NULL) THEN 'NO GROWTH'
        ELSE 'OTHERS' END AS responsable,
        dc.source_id AS customer_id,
        SUM(fs.gmv) AS gmv_usd,
        SUM((CASE WHEN (fsd.adjustment_reason NOT ILIKE '%merma%' AND fsd.adjustment_reason NOT ILIKE '%acmk%' AND fsd.adjustment_reason NOT ILIKE '%liquidation%') THEN fsd.product_discount ELSE 0 END)*cr.coefficient) AS discount_applied,
        100.00*discount_applied/gmv_usd AS per_dct,
        (1-(SUM(cogs_p_mtd*fs.product_quantity_x_step_unit)/SUM(fs.product_price*fs.product_quantity_x_step_unit)))*100.00 AS margin,
        gmv_usd*margin/100.00 AS cash_margin,
        cash_margin-discount_applied AS net_cash_margin,
        100.00*net_cash_margin/gmv_usd AS net_margin
    FROM dpr_sales.fact_sales                       fs
    INNER JOIN dpr_shared.dim_customer              dc  ON dc.customer_id = fs.dim_customer
    INNER JOIN dpr_shared.dim_site                  s   ON s.site_id = fs.dim_site
    INNER JOIN dpr_shared.dim_product               dp  ON dp.product_id = fs.dim_product
    INNER JOIN dpr_shared.dim_currency_conversion   cr  ON cr.currency_id = fs.dim_currency AND cr.start_date_id = 20180101
    LEFT JOIN dpr_sales.fact_sales_discounts        fsd ON fs.order_item_id = fsd.order_item_id 
    INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
    INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
    LEFT JOIN dpr_cross_business.fact_cross_business_insights m ON m.dim_stock_unit = sup.stock_unit_id AND m.dim_date = fs.dim_submitted_date

    WHERE 
        fs.gmv_enabled = TRUE
        AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
        AND fs.fb_order_status_id  IN (1,6,7,8)
        AND fs.is_deleted = FALSE
        AND fs.dim_status = 1
        AND dp.is_slot = 'false'
        AND fs.gmv_pxq_local > 0
        AND cogs_p_mtd > 0
        AND fs.order_close_date >= DATE_TRUNC('month', date(getdate()) - interval '{number} month')
        AND fs.order_close_date < DATE_TRUNC('month', date(getdate()) - interval '{number2} month')
    GROUP BY 1,2,3,4,5,6
    """.format(number=num, number2=num-1)
    df_gmv = run_read_dwd_query(query)
    df_gmv[["gmv_usd","discount_applied"]] = df_gmv[["gmv_usd","discount_applied"]].astype(float)

    return df_gmv

In [6]:
# Create an empty list to store DataFrames
dfs = []

# Iterate through the range and append each result to the list
for i in range(0, 2):
    df_gmv = gmv(i)
    dfs.append(df_gmv)
    print(i)
    
# Concatenate the list of DataFrames into a single DataFrame
result_df = pd.concat(dfs, ignore_index=True)

0
1


In [7]:
result_df.month.unique()

array([6, 5])

In [8]:
df2 =  pd.merge(result_df, df_kam, left_on=['city','weekk','customer_id'], right_on=['region_code','week','customer_id'], how='left')
df2 =  pd.merge(df2, df_fmg_comex, left_on=['city','weekk','customer_id'], right_on=['region_code','week_list','customer_id'], how='left')
df2 =  pd.merge(df2, df_fmg_br, left_on=['weekk','customer_id'], right_on=['week','customer_id'], how='left')

In [9]:
df_aux = df2[['customer_id','weekk','year_x']].drop_duplicates()

In [10]:
# Initialize 'is_graduated' column with 0
df_aux['is_graduated'] = 0

# Iterate over each row in df_aux
for index, row in df_aux.iterrows():
    i+=1
    customer_id = row['customer_id']
    week = row['weekk']
    year = row['year_x']

    # Check if customer_id is in df_grads
    if customer_id in df_grads['uid'].values:
        # Get relevant information from df_grads for the current customer
        customer_grads = df_grads[df_grads['uid'] == customer_id].iloc[0]

        # Experimental condition
        exp_condition = (
            (week >= customer_grads['week_grad']) & (year == customer_grads['year_grad']) or
            (year > customer_grads['year_grad'])
        )

        # Desgrad condition
        desgrad_condition = (
            (week >= customer_grads['week_grad']) & (year == customer_grads['year_grad']) or
            (year > customer_grads['year_grad'])
        ) and (
            (week <= customer_grads['week_desgrad']) & (year == customer_grads['year_desgrad']) or
            (year < customer_grads['year_desgrad'])
        )

        # Update 'is_graduated' column for the current row and week
        df_aux.loc[index, 'is_graduated'] = int(exp_condition or desgrad_condition)
        print(i)

3
4
5
6
8
9
13
14
15
17
18
19
21
23
24
25
26
27
28
29
32
34
35
38
43
46
47
49
51
53
54
59
60
62
64
65
66
67
70
71
72
73
76
77
78
80
82
83
86
88
89
90
91
92
93
94
95
96
97
98
102
104
105
111
112
113
115
118
120
121
122
123
125
126
127
128
130
131
132
134
138
139
140
141
144
146
147
148
150
151
153
154
155
157
158
159
160
162
163
164
165
167
169
170
173
176
178
179
180
181
182
184
185
187
188
189
190
191
194
196
197
198
200
201
202
204
205
206
209
212
213
214
215
217
221
222
223
225
226
229
230
231
232
235
237
238
239
241
243
244
246
248
253
254
255
256
257
259
260
261
263
264
266
267
268
269
270
271
274
275
276
279
280
281
282
283
284
286
290
291
292
293
294
295
297
298
299
300
303
304
305
307
308
315
317
318
319
321
323
324
325
326
328
329
331
332
334
335
339
340
341
344
345
346
347
349
350
351
352
354
355
356
357
359
361
362
366
370
371
376
378
379
380
381
382
383
384
387
388
389
390
391
392
393
394
397
399
400
401
404
406
407
408
409
410
411
413
414
415
417
419
422
423
425
428
434
43

In [11]:
df2 =  pd.merge(df2, df_aux, left_on=['weekk','customer_id','year_x'], right_on=['weekk','customer_id','year_x'], how='left')

In [12]:
df2.columns

Index(['city', 'year_x', 'month', 'weekk', 'responsable', 'customer_id',
       'gmv_usd', 'discount_applied', 'per_dct', 'margin', 'cash_margin',
       'net_cash_margin', 'net_margin', 'region_code_x', 'week_x', 'is_kam',
       'year_list', 'week_list', 'region_code_y', 'is_farming_x', 'week_y',
       'year_y', 'is_farming_y', 'is_graduated'],
      dtype='object')

In [13]:
df3 = df2[['city', 'month', 'weekk', 'responsable', 'customer_id', 'gmv_usd', 
     'discount_applied', 'per_dct', 'margin', 'cash_margin',
       'net_cash_margin', 'net_margin', 'is_kam', 'is_farming_x', 'is_farming_y', 'is_graduated']].copy()

In [14]:
df3 = df3.fillna(0)

In [15]:
df3['check'] = df3.is_farming_x+df3.is_farming_y+df3.is_graduated+df3.is_kam

In [16]:
df3.check.value_counts()

1.0    213066
0.0     55474
2.0      5895
3.0         5
Name: check, dtype: int64

In [17]:
def determine_group(row):
    if row['check'] == 1:
        if row['is_farming_x'] == 1 or row['is_farming_y'] == 1:
            return 'farming'
        elif row['is_kam'] == 1:
            return 'kam'
        elif row['is_graduated'] == 1:
            return 'grad'
    elif row['check'] == 0:
        return 'other'
    elif row['check'] == 2:
        if row['is_kam'] == 1:
            return 'kam'
        elif row['is_graduated'] == 1:
            return 'grad'
        else:
            return 'farming'
    elif row['check'] == 3:
        if row['is_kam'] == 1:
            return 'kam'
        elif row['is_graduated'] == 1:
            return 'grad'
        else:
            return 'farming'
    else:
        return np.nan

# Apply the custom function to create the 'group' column
df3['group'] = df3.apply(determine_group, axis=1)

In [18]:
df3.month.unique()

array([6, 5])

In [19]:
df4 = df3.groupby(['city','month','weekk','responsable','group']).agg({'gmv_usd':np.sum,'discount_applied':np.sum, 'cash_margin':np.sum, 'net_cash_margin':np.sum}).reset_index()

In [20]:
# Assuming your DataFrame is named 'df4'
df4['country'] = df4['city'].map({
    'BOG': 'CO',
    'BAQ': 'CO',
    'MDE': 'CO',
    'SPO': 'BR',
    'BHZ': 'BR',
    'CWB': 'BR',
    'VCP': 'BR',
    'CMX': 'MX'
})

# Create a new DataFrame for country-level aggregation
country_df4 = df4.groupby(['country', 'month','weekk', 'responsable', 'group']).agg({
    'gmv_usd': 'sum',
    'discount_applied': 'sum',
    'cash_margin':np.sum, 
    'net_cash_margin':np.sum
}).reset_index()
country_df4.rename(columns={'country': 'city'}, inplace=True)

# Create a DataFrame for LATAM-level aggregation
latam_df4 = df4.groupby(['month','weekk', 'responsable', 'group']).agg({
    'gmv_usd': 'sum',
    'discount_applied': 'sum',
    'cash_margin':np.sum, 
    'net_cash_margin':np.sum
}).reset_index()
latam_df4['city'] = 'LATAM'

df4.drop(['country'], axis=1, inplace=True)

In [21]:
# Append the aggregated country-level data to the original DataFrame
df4 = pd.concat([df4, country_df4], ignore_index=True, sort=False)

# Append the aggregated LATAM-level data to the original DataFrame
df4 = pd.concat([df4, latam_df4[['city','month','weekk', 'responsable', 'group', 'gmv_usd', 'discount_applied','cash_margin','net_cash_margin']]], ignore_index=True, sort=False)

# Sort the DataFrame based on the original index to preserve the order
df4.sort_index(inplace=True)

In [22]:
# Create a DataFrame for 'responsable' named 'ALL'
all_df = df4.groupby(['month','weekk', 'responsable', 'city']).agg({
    'gmv_usd': 'sum',
    'discount_applied': 'sum',
    'cash_margin':np.sum, 
    'net_cash_margin':np.sum
}).reset_index()
all_df['group'] = 'ALL'

# Append the aggregated 'ALL' data to the original DataFrame
df4 = pd.concat([df4, all_df[['city','month','weekk', 'responsable', 'group', 'gmv_usd', 'discount_applied','cash_margin','net_cash_margin']]], ignore_index=True, sort=False)

# Sort the DataFrame based on the original index to preserve the order
df4.sort_index(inplace=True)

In [23]:
# Create a DataFrame for 'responsable' named 'ALL'
all_df = df4.groupby(['month','weekk', 'group', 'city']).agg({
    'gmv_usd': 'sum',
    'discount_applied': 'sum',
    'cash_margin':np.sum, 
    'net_cash_margin':np.sum
}).reset_index()
all_df['responsable'] = 'ALL'

# Append the aggregated 'ALL' data to the original DataFrame
df4 = pd.concat([df4, all_df[['city','month','weekk', 'responsable', 'group', 'gmv_usd', 'discount_applied','cash_margin','net_cash_margin']]], ignore_index=True, sort=False)

# Sort the DataFrame based on the original index to preserve the order
df4.sort_index(inplace=True)

In [24]:
df4

,city,month,weekk,responsable,group,gmv_usd,discount_applied,cash_margin,net_cash_margin
0,BHZ,5,18,ADDITIONAL INVESTMENT,farming,273.98121,20.909474,100.3079381,79.398463997600
1,BHZ,5,18,ADDITIONAL INVESTMENT,grad,2036.50817,157.164214,720.9440321,563.779818430400
2,BHZ,5,18,ADDITIONAL INVESTMENT,kam,587.94305,47.000001,205.3609154,158.360914460000
3,BHZ,5,18,ADDITIONAL INVESTMENT,other,61.47916,5.981053,24.5195708,18.538518048800
4,BHZ,5,18,HOOK,farming,15399.29614,424.545272,1397.9269581,973.381686451200
...,...,...,...,...,...,...,...,...,...
1604,BR,6,25,ALL,other,72193.72497,623.234749,12126.4794917,11503.244742393200
1605,CWB,6,25,ALL,other,3607.78383,15.450527,591.1884340,575.737907375200
1606,LATAM,6,25,ALL,other,72193.72497,623.234749,12126.4794917,11503.244742393200
1607,SPO,6,25,ALL,other,53702.01357,505.516852,9281.1225594,8775.605707184400


In [25]:
# Assuming your DataFrame is named 'df'
df4['total_gmv'] = df4.groupby(['city', 'month','weekk', 'group'])['gmv_usd'].transform('sum')
df4['total_gmv'] = df4['total_gmv']/2
df4['discount_percentage'] = (df4['discount_applied'] / df4['total_gmv']) * 100

In [26]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1609 entries, 0 to 1608
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   city                 1609 non-null   object 
 1   month                1609 non-null   int64  
 2   weekk                1609 non-null   int64  
 3   responsable          1609 non-null   object 
 4   group                1609 non-null   object 
 5   gmv_usd              1609 non-null   float64
 6   discount_applied     1609 non-null   float64
 7   cash_margin          1609 non-null   object 
 8   net_cash_margin      1609 non-null   object 
 9   total_gmv            1609 non-null   float64
 10  discount_percentage  1609 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 138.4+ KB


In [27]:
paramss_csv=GoogleSheet("1pVqWRHSn3qFtyrCz65GWFHvI2aHT1zDgqz8gP0MbmBo")
city_paramss = paramss_csv.get_as_dataframe('INFO')

In [28]:
paramss_csv.set_with_dataframe(df4, worksheet='INFO', autocreate=True)

In [29]:
#df4.to_excel('aaaa.xlsx')